<a href="https://colab.research.google.com/github/Zinga18018/Ipynb_Project_Snippets/blob/main/Spam_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV,train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegressionCV
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score,matthews_corrcoef,classification_report,confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
df = pd.read_csv('/content/mail_data.csv')

In [7]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [9]:
label_encoder = OneHotEncoder()
df['Category'] = label_encoder.fit_transform(df['Category'].values.reshape(-1, 1)).toarray()

In [10]:
df.head()

,Category,Message
0,1.0,"Go until jurong point, crazy.. Available only ..."
1,1.0,Ok lar... Joking wif u oni...
2,0.0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1.0,U dun say so early hor... U c already then say...
4,1.0,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
def clean_text(text):
  text = text.lower()
  text_tokens = nltk.word_tokenize(text)
  text_tokens = [word for word in text_tokens if word not in stopwords.words('english')]
  lemmatizer = WordNetLemmatizer()
  text_tokens = [lemmatizer.lemmatize(word) for word in text_tokens]
  return ' '.join(text_tokens)

In [12]:

df['cleaned_message'] = df['Message'].apply(lambda x: ' '.join(clean_text(x)))


X = df['cleaned_message']
y = df['Category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (4457,)
Shape of X_test: (1115,)
Shape of y_train: (4457,)
Shape of y_test: (1115,)


In [15]:
# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print(f"Shape of X_train_tfidf: {X_train_tfidf.shape}")
print(f"Shape of X_test_tfidf: {X_test_tfidf.shape}")

Shape of X_train_tfidf: (4457, 5000)
Shape of X_test_tfidf: (1115, 5000)


In [20]:
# Initialize and train a Logistic Regression model with cross-validation
logistic_model = LogisticRegressionCV(cv=5, random_state=42, solver='liblinear', scoring='f1', n_jobs=-1)
logistic_model.fit(X_train_tfidf, y_train)

LogisticRegressionCV(cv=5, n_jobs=-1, random_state=42, scoring='f1',
                     solver='liblinear')

In [21]:
# Make predictions on the test set
y_pred_logistic = logistic_model.predict(X_test_tfidf)

# Evaluate the model
f1 = f1_score(y_test, y_pred_logistic)
mcc = matthews_corrcoef(y_test, y_pred_logistic)

print(f"Logistic Regression F1-Score: {f1:.4f}")
print(f"Logistic Regression Matthews Correlation Coefficient: {mcc:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred_logistic))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_logistic))

Logistic Regression F1-Score: 0.9871
Logistic Regression Matthews Correlation Coefficient: 0.9012

Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.89      0.91       149
         1.0       0.98      0.99      0.99       966

    accuracy                           0.98      1115
   macro avg       0.96      0.94      0.95      1115
weighted avg       0.98      0.98      0.98      1115


Confusion Matrix:
 [[132  17]
 [  8 958]]


In [22]:
print(df['Category'].value_counts())
print(df['Category'].value_counts(normalize=True))

Category
1.0    4825
0.0     747
Name: count, dtype: int64
Category
1.0    0.865937
0.0    0.134063
Name: proportion, dtype: float64
